# Welcome to introduction to dashboards with Plotly and Dash

-------------------------------------------------------------------------------------------------------------------------------


### Workshop facilitators: Laura Gutierrez Funderburk, Hanh Tong

### About this workshop

In this workshop we will explore some characteristics of the housing market in Canada. 

It is important to note that this workshop assumes:

1. Data cleaning and exploration was completed prior to developing the dashboard
2. Some comfort with `pandas` and visualization is assumed
3. Comfort navigating the Jupyter environment is needed



### Workshop schedule:
-------------------------------------------------------------------------------------------------------------------------------


#### 1. Part I: Data exploration

In this section, we will first spend time getting familiar with the data. We will use the `pandas` and `plotly` libraries, we will also explore the `DEX` feature within Noteable to ease getting a good sense for what the data contains.

In this section, we will also explore the notion of factoring code into functions, and the notion of writing a Python script that we can use to easily recreate our results. 

#### 2. Part II: Dashboard components

In this section, we will take what we built together in part I and explore the main components in a Dash dashboard. 

## Part II: Dashboard components
-------------------------------------------------------------------------------------------------------------------------------


The four main components in a Dash dashboard are:

1. A .py script (also known as your dashboard app) `app.py`

2. A requirements.txt file with the Python dependencies for your app

3. A Procfile

4. A .gitignore file


### The .py script anatomy

-------------------------------------------------------------------------------------------------------------------------------


The anatomy of a .py script is as follows:

```python
import math

def function1(par1: int, par2: str) -> None:
    """
    Parameters
    ----------
        par1: (dataframe object) reshaped data frame object with mortage, delinquency and population data
        par2: (string) "box", "violin", "scatter", "line"
         
    Returns:
    --------
        None
    """
    print(f"{par1} is spelled '{par2}'")
        
if __name__ == '__main__':  
    
    # Code that does something
    function1(1, "one")
```

### A running example

-------------------------------------------------------------------------------------------------------------------------------

Let's package the content of the function `graph_region` that we crafted in part I. The script looks as follows:

In [ ]:
# %load ./nb-scripts/base_script.py
import pandas as pd
import plotly.express as px

def graph_region(region_df, graph_type: str, dimension1: str, dimension2: str):
    """
    Parameters
    ----------
        region_df: (dataframe object) reshaped data frame object with mortage, delinquency and population data
        graph_type: (string) "box", "violin", "scatter", "line"
        dimension1: (str) one of 'Time' or 'Geography'
        dimension2: (str) one of 'AverageMortgageAmount', 'AverageMortgageAmount' or 'PopulationSize'
        
    Returns:
    --------
        Plotly figure
    """
    
    plot_dict = {'box': px.box,'violin': px.violin, 'scatter': px.scatter, 'line':px.line}
        
    try:
        # Initialize function
        fig = plot_dict[graph_type](region_df, 
                                    x=dimension1, 
                                    y=dimension2, 
                                    color = "Geography",
                                   hover_name = "Time")
        # Format figure 
        title_string = f'Chart: {graph_type} plot of {dimension1} and {dimension2} by Geography'
        fig.update_layout(title = title_string)
        fig.update_xaxes(tickangle=-45)
        return fig
    
    except KeyError:
        print("Key not found. Make sure that 'graph_type' is in ['box','violin', 'scatter', 'line']")
    except ValueError:
        print("Dimension is not valid. dimension1 is one of 'Time' or 'Geography'")
        print("dimension2 is one of 'AverageMortgageAmount', 'DelinquencyRate', 'PopulationSize'")
        
        
if __name__ == '__main__':  
    
    # Read the data into a dataframe 
    url = 'https://raw.githubusercontent.com/Vancouver-Datajam/dashboard-workshop-dash/main/data/delinquency_mortgage_population_2021_2020.csv'
    data_pop_del_mort_df = pd.read_csv(url, index_col=0)
    
    # See the first few rows
    display(data_pop_del_mort_df.head(10))
    
    # Visualize
    fig1 = graph_region(data_pop_del_mort_df, 'line', "Time", "AverageMortgageAmount")
    fig2 = graph_region(data_pop_del_mort_df, 'box', "Geography", "AverageMortgageAmount")
    fig3 = graph_region(data_pop_del_mort_df, 'scatter', "AverageMortgageAmount", "DelinquencyRate")
    
    fig1.show()
    fig2.show()
    fig3.show()

In [ ]:
%run -i ./nb-scripts/base_script.py


### What is wrong with the plots being laid out this way?
-------------------------------------------------------------------------------------------------------------------------------


A lot...

This is hard to read, hard to interact with, and overwhelming. 

Furthemore, we'd need to change the code every time we want to render a different plot. 

We introduce the app machinery.

### The `app.py` script
-------------------------------------------------------------------------------------------------------------------------------


The `app.py` script is a Python script responsible for deploying our app. It is just a .py script with additional machinery. 


### The `app.py` additional machinery
-------------------------------------------------------------------------------------------------------------------------------

Our app is going to contain mostly the same as a regular .py script, plus a few new items:

- Dash library imports
- External stylesheets
- App initialization
- App layout
- Decorator callbacks

### Dash library imports
-------------------------------------------------------------------------------------------------------------------------------

```python

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

```

### External stylesheets
-------------------------------------------------------------------------------------------------------------------------------

```python

# Stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


```

Dash applications are rendered in the web browser with CSS and JavaScript. On page load, Dash serves a small HTML template that includes references to the CSS and JavaScript that are required to render the application. You can use stylesheets in your dashboard, similarly to how you'd use them to style your website.

Read more https://dash.plotly.com/external-resources

### App initialization
-------------------------------------------------------------------------------------------------------------------------------

```python

# Intialize app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server


```


### App layout
-------------------------------------------------------------------------------------------------------------------------------

```python

app.layout = html.Div([sidebar, content])


```

This component describes what the application looks like.

https://dash.plotly.com/layout

### Decorator callbacks
-------------------------------------------------------------------------------------------------------------------------------

```python

@app.callback(
     Output('graph-id', 'figure'),
    Input('value-id', 'value')
def update_figure(parameter):
    """
    Parameters
    ----------
        parameter: This is the value that will be updated in the app
    """
    
    # Code that uses the variable value to make a change 
    ...
    # Code that generates figure based on value
    fig = ...

    return fig


```


Decorator callbacks are Python functions that are automatically called by Dash whenever an input component's property changes. This is what makes it possible for dashboards to be interactive.

Read more https://dash.plotly.com/basic-callbacks

Exit slideshow to see full script.

```python
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# ----------------------------------------------------------------------------------#
# Read data


# ----------------------------------------------------------------------------------#
# App section

# Stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Intialize app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

app.layout = # some code with our layout


@app.callback(
     Output('graph-id', 'figure'),
    Input('value-id', 'value')
def update_figure(parameter):
    """
    Parameters
    ----------
        parameter: This is the value that will be updated in the app
    """
    
    # Code that uses the variable value to make a change 
    ...
    # Code that generates figure based on value
    fig = ...

    return fig


if __name__ == '__main__':  
    app.run_server(debug=True) 
```



### Let's work together to turn our plotting script into an app using Dash

-------------------------------------------------------------------------------------------------------------------------------


In [ ]:

import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

def graph_region(region_df, graph_type: str, dimension1: str, dimension2: str) -> None:
    """
    Parameters
    ----------
        region_df: (dataframe object) reshaped data frame object with mortage, delinquency and population data
        graph_type: (string) "box", "violin", "scatter", "line"
        dimension1: (str) one of 'Time' or 'Geography'
        dimension2: (str) one of 'AverageMortgageAmount', 'AverageMortgageAmount' or 'PopulationSize'
        
    Returns:
    --------
        None
    """
    
    plot_dict = {'box': px.box,'violin': px.violin, 'scatter': px.scatter, 'line':px.line}
        
    try:
        # Initialize function
        fig = plot_dict[graph_type](region_df, 
                                    x=dimension1, 
                                    y=dimension2, 
                                    color = "Geography",
                                   hover_name = "Time")
        # Format figure 
        title_string = f'Chart: {graph_type} plot of {dimension1} and {dimension2} by Geography'
        fig.update_layout(title = title_string)
        fig.update_xaxes(tickangle=-45)
        fig.show()
    
    except KeyError:
        print("Key not found. Make sure that 'graph_type' is in ['box','violin', 'scatter', 'line']")
    except ValueError:
        print("Dimension is not valid. dimension1 is one of 'Time' or 'Geography'")
        print("dimension2 is one of 'AverageMortgageAmount', 'DelinquencyRate', 'PopulationSize'")
        
        
# ----------------------------------------------------------------------------------#
# Read data

url = 'https://raw.githubusercontent.com/Vancouver-Datajam/dashboard-workshop-dash/main/data/delinquency_mortgage_population_2021_2020.csv'
data_pop_del_mort_df = pd.read_csv(url, index_col=0)

# ----------------------------------------------------------------------------------#
# App section        
        

# Stylesheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Intialize app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

app.layout = html.Div([
                html.H1("Housing graphs"),
                dcc.Dropdown(id='province',
                             options=[{'label': i, 'value': i} for i in data_pop_del_mort_df['Geography'].unique()],
                             value= 'Newfoundland')
                     ])


        
if __name__ == '__main__':  
    
    app.run_server(debug=True) 

### Why choosing your layout before coding matters

-------------------------------------------------------------------------------------------------------------------------------

The previous cell renders a dashboard containing only a dropdown menu with provinces in Canada. 

Before we build our dashboard, we first need to spend a bit of time thinking about what it is our dashboard contains, and how it is laid out. 

Deciding a layout prior to coding will make cleaning and maintaining your app easier. 

You can use any software you like to create your diagrams. 

For this workshop we will use https://app.diagrams.net/

### A basic layout: a dropdown menu and a plot

-------------------------------------------------------------------------------------------------------------------------------

Our first step will be to implement a dropdown menu that selects one of: scatter plot, line chart and box chart.

https://drive.google.com/file/d/1JHjlf3BBDBMTS_N76xE5WZslOBOUzfW9/view?usp=sharing

### The dropdown menu 

-------------------------------------------------------------------------------------------------------------------------------

The code below will render a dropdown menu whose options are 'box', 'scatter', 'line', 'violin'.

If you are familiar with HTML, you will recognize a few important tags including `div` and `h1`. We can use `dash_core_components` with the alias `dcc` to access the `Dropdown feature`. 

The options are a list of dictionaries with labels and values.

```python
app.layout = html.Div([
             # This div contains a header H1, a dropdown to select the kind of plot and the plot
            html.H1("Different kinds of plots"),
            dcc.Dropdown(
                        id='graph-type',
                        options=[{'label': 'Scatter plot', 'value': 'scatter'},
                                {'label': 'Line plot', 'value': 'line'},
                                {'label': 'Box plot', 'value': 'box'}],
                        value= 'box')

        
])
```

In [ ]:
# Let's take a look at the labels

options=[{'label': i, 'value': i} for i in ['box', 'scatter', 'line', 'violin']]

options

### A plot that changes based on the plot type.

-------------------------------------------------------------------------------------------------------------------------------

The code below will render a dropdown menu whose options are determined by the `Geography` column in the `data_pop_del_mort_df` dataframe object. 

If you are familiar with HTML, you will recognize a few important tags including `div` and `h1`. We can use `dash_core_components` with the alias `dcc` to access the `Dropdown feature`. 

The options are a list of dictionaries with labels and values.

```python
app.layout = html.Div([
             # This div contains a header H1, a dropdown to select the kind of plot and the plot
            html.H1("Different kinds of plots"),
            dcc.Dropdown(
                        id='graph-type',
                        options=[{'label': 'Violin plot', 'value': 'violin'},
                                {'label': 'Box plot', 'value': 'box'}],
                        value= 'box'),
            dcc.Graph(id='graph-render')
])
 ```

### A mechanism to update the figure type

-------------------------------------------------------------------------------------------------------------------------------

```python

app.layout = html.Div([
             # This div contains a header H1, a dropdown to select the kind of plot and the plot
            html.H1("Different kinds of plots"),
            dcc.Dropdown(
                        id='graph-type',
                        options=[{'label': 'Violin plot', 'value': 'violin'},
                                {'label': 'Box plot', 'value': 'box'}],
                        value= 'box'),
            dcc.Graph(id='graph-render')
])

@app.callback(
    Output('graph-render', 'figure'),
    Input('graph-type', 'value'))
def update_figure0(selected_graph):
    filtered_df = data_pop_del_mort_df
    fig0 = graph_region(filtered_df, selected_graph, "Geography", "AverageMortgageAmount")
    return fig0

```

### A dropdown that updates two figures

-------------------------------------------------------------------------------------------------------------------------------


```python
app.layout = html.Div([
                html.Div([
                    # The province dropdown
                    html.H1("Housing graphs"),
                    dcc.Dropdown(
                        id='province',
                        options=[{'label': i, 'value': i} for i in data_pop_del_mort_df['Geography'].unique()],
                        value= 'Newfoundland'
                    )
                ]),
                
                # Plotting two graphs side by side
                html.Div([
                    html.Div([
                        dcc.Graph(id='graph-time-mortgage')
                    ], className="six columns"),

                    html.Div([
                        dcc.Graph(id='graph-time-del')
                    ], className="six columns"),
                ], className="row")
])

    
@app.callback(
    Output('graph-time-mortgage', 'figure'),
    Input('province', 'value'))
def update_figure1(selected_province):
    df = data_pop_del_mort_df
    filtered_df = df[df['Geography'] == selected_province]
    fig1 = graph_region(filtered_df, 'line', "Time", "AverageMortgageAmount")
    return fig1

@app.callback(
    Output('graph-time-del', 'figure'),
    Input('province', 'value'))
def update_figure2(selected_province):
    df = data_pop_del_mort_df
    filtered_df = df[df['Geography'] == selected_province]  
    fig2 = graph_region(filtered_df, 'line', "Time", "DelinquencyRate")
    return fig2
```

### Several dropdown menus updating the same figure

-------------------------------------------------------------------------------------------------------------------------------


```python

text_style = {
    'textAlign' : 'center',
    'color' : "black"
}

card_text_style = {
    'textAlign' : 'center',
    'color' : 'black'
}
    
app.layout = html.Div([
    html.Div([
        html.H2("Housing Market Trends in Vancouver (quarterly, 2012 - 2020)", style=card_text_style),
        html.Div([
            
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': 'Geography', 'value': 'Geography'},
                         {'label': 'Time', 'value': 'Time'},
                        {'label': 'Population Size', 'value': 'PopulationSize'},
                         {'label': 'Delinquency Rate', 'value': 'DelinquencyRate'},
                        {'label': 'Average Mortgage Amount', 'value': 'AverageMortgageAmount'}],
                value='Geography'
            ),
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': 'Population Size', 'value': 'PopulationSize'},
                         {'label': 'Delinquency Rate', 'value': 'DelinquencyRate'},
                        {'label': 'Average Mortgage Amount', 'value': 'AverageMortgageAmount'}],
                value='PopulationSize'
            ),
            
        ]),
        
        html.Div([
            dcc.Checklist(
                id='graph-type',
                options=[{'label': 'Violin plot', 'value': 'violin'},
                         {'label': 'Box plot', 'value': 'box'},
                        {'label': 'Scatter plot', 'value': 'scatter'},
                        {'label': 'Line plot', 'value': 'line'}],
                value=['violin']
            )
        ])
    ]),

    dcc.Graph(id='indicator-graphic'),

    
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('graph-type', 'value'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'))
def update_figure3(selected_graph, xaxis, yaxis):
    filtered_df = data_pop_del_mort_df
    fig3 = graph_region(filtered_df, selected_graph[0], xaxis, yaxis)
    return fig3
```

### Hosting your Dashboard

-------------------------------------------------------------------------------------------------------------------------------

Once you are happy and have tested your dashboard locally, we need to store our app in a GitHub repository with the following files:

Procfile

requirements.txt

.gitignore

We created a base repository with all these files for you. All you need to do is:

0. Press the "Template" button in https://github.com/Vancouver-Datajam/dashboard-prep to create your own copy. 

1. Clone your copy of the repository on your local computer

2. Add the app.py filename onto your local copy of the repository.

3. Push app.py to `origin/main`

4. Login to your Heroku account via the terminal.

6. Run `git remote -v` to confirm 

5. Run `git add .` 

6. Run `git push origin/heroku`

7. Visit the page to ensure it is up or troubleshoot until you find the error.